In [1]:
import pandas as pd
import re

# ==========================
# CONFIG
# ==========================

input_file = "tb.xlsx"
degree_section = "BSIT-6th-M1"
output_file = "Filtered_Timetable.xlsx"

# ==========================
# LOAD FILE
# ==========================

df = pd.read_excel(input_file, header=1)
df = df.dropna(axis=1, how='all')

df.rename(columns={
    df.columns[0]: "Day",
    df.columns[1]: "Room"
}, inplace=True)

# Fix merged Day cells
df["Day"] = df["Day"].ffill()

# Clean Day column
df["Day"] = df["Day"].astype(str).str.strip()

# Remove Break column if exists
df = df.loc[:, ~df.columns.str.contains("Break", case=False, na=False)]

# ==========================
# MELT
# ==========================

df_long = df.melt(
    id_vars=["Day", "Room"],
    var_name="Time Slot",
    value_name="Lecture Info"
)

df_long = df_long.dropna(subset=["Lecture Info"])
df_long["Lecture Info"] = df_long["Lecture Info"].astype(str)

# ==========================
# FILTER SECTION
# ==========================

filtered = df_long[
    df_long["Lecture Info"].str.contains(degree_section, case=False, na=False)
].copy()

# ==========================
# SAFE DAY SORTING
# ==========================

day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

# Normalize for matching (without deleting)
filtered["Day_Clean"] = (
    filtered["Day"]
    .str.replace(":", "", regex=False)
    .str.strip()
    .str.title()
)

filtered["Day_Clean"] = pd.Categorical(
    filtered["Day_Clean"],
    categories=day_order,
    ordered=True
)

# ==========================
# TIME SORTING
# ==========================

def extract_start_time(t):
    match = re.search(r"(\d+):(\d+)", str(t))
    if match:
        return int(match.group(1)) * 60 + int(match.group(2))
    return 0

filtered["Sort_Time"] = filtered["Time Slot"].apply(extract_start_time)

filtered = filtered.sort_values(by=["Day_Clean", "Sort_Time"])

# Final clean columns
filtered = filtered[["Day", "Time Slot", "Room", "Lecture Info"]]

# ==========================
# EXPORT
# ==========================

filtered.to_excel(output_file, index=False)

print("Filtered timetable saved as:", output_file)

Filtered timetable saved as: Filtered_Timetable.xlsx


working code for schedule filter

In [3]:
import pandas as pd
import re
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

# ==========================
# CONFIG  ← only change these
# ==========================
input_file    = "tb.xlsx"
degree_section = "BSIT-6th-M2"
output_file   = "Filtered_Timetable.xlsx"

# ==========================
# LOAD & CLEAN
# ==========================
df = pd.read_excel(input_file, header=1)
df = df.dropna(axis=1, how='all')
df.columns = ['Day', 'Room'] + list(df.columns[2:])
df['Day'] = df['Day'].ffill().astype(str).str.strip()

# Expand abbreviated day names to full names
day_map = {
    'Mon': 'Monday', 'Tue': 'Tuesday', 'Wed': 'Wednesday',
    'Thu': 'Thursday', 'Fri': 'Friday', 'Sat': 'Saturday'
}
df['Day'] = df['Day'].map(day_map).fillna(df['Day'])

# Drop the break/lunch column
df = df.loc[:, ~df.columns.str.fullmatch(r'1:00-2:00', na=False)]

# ==========================
# MELT & FILTER
# ==========================
df_long = df.melt(id_vars=['Day', 'Room'], var_name='Time Slot', value_name='Lecture Info')
df_long = df_long.dropna(subset=['Lecture Info'])
df_long['Lecture Info'] = df_long['Lecture Info'].astype(str).str.strip()

filtered = df_long[
    df_long['Lecture Info'].str.contains(degree_section, case=False, na=False)
].copy()

# ==========================
# SORT
# ==========================
def extract_start_time(t):
    m = re.search(r'(\d+):(\d+)', str(t))
    return int(m.group(1)) * 60 + int(m.group(2)) if m else 0

def extract_end_time(t):
    matches = re.findall(r'(\d+):(\d+)', str(t))
    return int(matches[-1][0]) * 60 + int(matches[-1][1]) if matches else 0

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
filtered['Day'] = pd.Categorical(filtered['Day'], categories=day_order, ordered=True)
filtered['Sort_Time'] = filtered['Time Slot'].apply(extract_start_time)
filtered = filtered.sort_values(['Day', 'Room', 'Lecture Info', 'Sort_Time']).reset_index(drop=True)

# ==========================
# DEDUPLICATE
# Each class spans 2 consecutive time slots in the Excel — merge them into one row
# ==========================
def merge_slots(group):
    group = group.sort_values('Sort_Time').reset_index(drop=True)
    rows = []
    i = 0
    while i < len(group):
        start_slot = group.loc[i, 'Time Slot']
        start_time = group.loc[i, 'Sort_Time']
        end_slot   = start_slot
        j = i + 1
        while j < len(group):
            if abs(group.loc[j, 'Sort_Time'] - extract_end_time(end_slot)) <= 10:
                end_slot = group.loc[j, 'Time Slot']
                j += 1
            else:
                break
        if start_slot == end_slot:
            time_range = start_slot.strip()
        else:
            t_start = re.search(r'^[\d:]+', start_slot.strip()).group()
            t_end   = re.search(r'[\d:]+$', end_slot.strip()).group()
            time_range = f"{t_start} - {t_end}"
        rows.append({'Time Slot': time_range, 'Sort_Time': start_time})
        i = j
    return pd.DataFrame(rows)

result_parts = []
for (day, room, lecture), group in filtered.groupby(['Day', 'Room', 'Lecture Info'], observed=True):
    merged = merge_slots(group)
    merged['Day']          = day
    merged['Room']         = room
    merged['Lecture Info'] = lecture
    result_parts.append(merged)

result = pd.concat(result_parts, ignore_index=True)
result['Day'] = pd.Categorical(result['Day'], categories=day_order, ordered=True)
result = result.sort_values(['Day', 'Sort_Time']).reset_index(drop=True)

# ==========================
# PARSE LECTURE INFO
# ==========================
def parse_lecture(info):
    lines    = [l.strip() for l in info.split('\n') if l.strip()]
    subject  = lines[0] if lines else ''
    sections = ', '.join(l for l in lines[1:] if l.startswith('BS'))
    teacher  = next((l for l in lines[1:] if not l.startswith('BS')), '')
    return subject, sections, teacher

result[['Subject', 'Section(s)', 'Teacher']] = result['Lecture Info'].apply(
    lambda x: pd.Series(parse_lecture(x))
)

final = result[['Day', 'Time Slot', 'Room', 'Subject', 'Section(s)', 'Teacher']].copy()
final['Room'] = final['Room'].str.replace('\n', ' ').str.strip()
final['Day']  = final['Day'].astype(str)

# ==========================
# STYLED EXCEL OUTPUT
# ==========================
wb = Workbook()
ws = wb.active
ws.title = "Timetable"

# Title row
ws.merge_cells('A1:F1')
ws['A1'] = f"Timetable – {degree_section} | Spring Semester 2026"
ws['A1'].font      = Font(name='Arial', bold=True, size=13, color='FFFFFF')
ws['A1'].fill      = PatternFill('solid', start_color='1F4E79')
ws['A1'].alignment = Alignment(horizontal='center', vertical='center')
ws.row_dimensions[1].height = 28

# Header row
headers    = ['Day', 'Time Slot', 'Room', 'Subject', 'Section(s)', 'Teacher']
col_widths = [12,    18,          22,     16,         22,           28]
for col, (h, w) in enumerate(zip(headers, col_widths), 1):
    cell            = ws.cell(row=2, column=col, value=h)
    cell.font       = Font(name='Arial', bold=True, color='FFFFFF', size=11)
    cell.fill       = PatternFill('solid', start_color='2E75B6')
    cell.alignment  = Alignment(horizontal='center', vertical='center')
    ws.column_dimensions[cell.column_letter].width = w

# Per-day background colors
day_colors = {
    'Monday':    'DDEEFF',
    'Tuesday':   'DDF0DD',
    'Wednesday': 'FFF2CC',
    'Thursday':  'FCE4D6',
    'Friday':    'EAD1DC',
    'Saturday':  'E2EFDA',
}

thin   = Side(style='thin', color='AAAAAA')
border = Border(left=thin, right=thin, top=thin, bottom=thin)

prev_day = None
for r, row in final.iterrows():
    excel_row = r + 3
    day       = row['Day']
    bg        = day_colors.get(day, 'FFFFFF')

    for col, val in enumerate(row, 1):
        cell            = ws.cell(row=excel_row, column=col, value=val)
        cell.font       = Font(name='Arial', size=10)
        cell.fill       = PatternFill('solid', start_color=bg)
        cell.alignment  = Alignment(horizontal='left', vertical='center', wrap_text=True)
        cell.border     = border

    day_cell = ws.cell(row=excel_row, column=1)
    if day == prev_day:
        day_cell.value = ''          # hide repeated day name
    else:
        day_cell.font = Font(name='Arial', size=10, bold=True)
    prev_day = day

    ws.row_dimensions[excel_row].height = 30

wb.save(output_file)
print(f"✅ Saved: {output_file}  ({len(final)} classes found)")
print(final[['Day', 'Time Slot', 'Subject', 'Teacher']].to_string(index=False))

✅ Saved: Filtered_Timetable.xlsx  (8 classes found)
      Day     Time Slot                          Subject                           Teacher
   Monday   8:00 - 9:40 IT-508-P BSIT-6th-M1 BSIT-6th-M2                 Ms. Rizwana Usman
   Monday  9:40 - 11:20             IT-512-T BSIT-6th-M2                   Ms Iram Shazadi
  Tuesday  8:50 - 10:30             IT-506-T BSIT-6th-M2                      Mr.Ali Usama
Wednesday 10:30 - 12:10 IT-510-P BSIT-6th-M1 BSIT-6th-M2 Dr. Syed Mushhad Mustuzhar Gilani
 Thursday   1:10 - 2:50 IT-504-T BSIT-6th-M1 BSIT-6th-M2                         Sana Azam
 Thursday   8:00 - 9:40             IT-512-P BSIT-6th-M2                   Ms Iram Shazadi
 Thursday  9:40 - 11:20 IT-510-T BSIT-6th-M1 BSIT-6th-M2 Dr. Syed Mushhad Mustuzhar Gilani
 Thursday  11:20 - 1:00 IT-508-T BSIT-6th-M1 BSIT-6th-M2                 Ms. Rizwana Usman


In [17]:
import re

text = "My phone number is 123-456-7890"
match = re.search(r'\d{3}-\d{3}-\d{4}', text)
if match:
    print(match.group())  # Output: 123-456-7890

123-456-7890


In [9]:
def calculate(a,b):
    return a+b
result=calculate(12,12)
print(result)
print(result*3)

24
72


In [ ]:
import fitz
import re
import csv

def extract_to_csv(pdf_path, output_csv):
    pdf = fitz.open(pdf_path)
    rows = []

    for page_num in range(len(pdf)):
        text = pdf[page_num].get_text()

        date     = re.search(r'Date:\s*(\d{2}/\d{2}/\d{4})', text)
        trans_id = re.search(r'(ch_[A-Za-z0-9]+)', text)
        total    = re.search(r'Balance\s+(USD \$[\d.]+)', text)

        rows.append({
            "Page"           : page_num + 1,
            "Date"           : date.group(1) if date else "N/A",
            "Transaction ID" : trans_id.group(1) if trans_id else "N/A",
            "Total"          : total.group(1) if total else "N/A"
        })

    with open(output_csv, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["Page", "Date", "Transaction ID", "Total"])
        writer.writeheader()
        writer.writerows(rows)

    print(f"✅ Saved to {output_csv}")

extract_to_csv("Downloadable-PDF-Invoices-Add-On-Samples.pdf", "invoices.csv")